In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

In [3]:
df = pd.read_excel(r"C:\Users\Shibbs\Desktop\Praxis\CAPP\cwd\datasets\final\df.xlsx")

In [44]:
#df=pd.read_excel(r"C:\Users\Shibbs\Desktop\groupby.xlsx")

In [4]:
df.columns

Index(['Unnamed: 0', 'event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885124 entries, 0 to 885123
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     885124 non-null  int64  
 1   event_time     885124 non-null  object 
 2   event_type     885124 non-null  object 
 3   product_id     885124 non-null  int64  
 4   category_id    885124 non-null  float64
 5   category_code  731308 non-null  object 
 6   brand          672760 non-null  object 
 7   price          885124 non-null  float64
 8   user_id        885124 non-null  float64
 9   user_session   884959 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 67.5+ MB


In [6]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [7]:
df.category_id=df.category_id.astype('object')

In [8]:
df.category_id.nunique()

718

In [9]:
df.head(10)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-11-20 15:19:47 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,9cdf5Yzi4W
1,2020-10-18 18:29:34 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,DomDtlfN2x
2,2020-11-20 19:18:47 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,N5U9nr75XP
3,2020-12-20 01:25:28 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,apG36IgOcf
4,2021-02-24 15:59:50 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,7CZ3WuPoU3
5,2021-01-12 12:57:43 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,bVBY4mrES8
6,2021-02-24 15:58:41 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,7CZ3WuPoU3
7,2020-11-08 19:52:34 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,BoOX5GqMUb
8,2021-02-24 15:58:12 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,7CZ3WuPoU3
9,2021-01-16 16:45:09 UTC,view,1642613,2144415938450030080.0,auto.accessories.immobilizer,falcon,25.56,1.515916e+18,86v346hK2d


In [10]:
df['user_score'] = df['event_type'].map({'view':1,'cart':16,'purchase':48})
df['user_purchase'] = df['event_type'].apply(lambda x: 1 if x=='purchase' else 0)
df['price_category'] = 0
j=0
# divide price range into 5 categories (Low to High) for each category code.
for i in df['category_id'].unique():
    # find all indices of given unique category_code and fill the price_category column of respective indices with 5 category range of price.
    df.loc[df['category_id']==i,'price_category'] = pd.qcut(x=df['price'][df['category_id']==i],q=3,labels=False, duplicates = 'drop')
    j=j+1
    print(j,sep=' ',end=' ',flush=True)

df.price_category=df.price_category+1
df=df[['event_time','event_type','product_id','category_id','price','user_id','user_session','user_score','user_purchase','price_category']]
df.dropna(inplace=True)
df.head()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

,event_time,event_type,product_id,category_id,price,user_id,user_session,user_score,user_purchase,price_category
0,2020-11-20 15:19:47 UTC,view,1642613,2144415938450030080.0,25.56,1.515916e+18,9cdf5Yzi4W,1,0,2.0
1,2020-10-18 18:29:34 UTC,view,1642613,2144415938450030080.0,25.56,1.515916e+18,DomDtlfN2x,1,0,2.0
2,2020-11-20 19:18:47 UTC,view,1642613,2144415938450030080.0,25.56,1.515916e+18,N5U9nr75XP,1,0,2.0
3,2020-12-20 01:25:28 UTC,view,1642613,2144415938450030080.0,25.56,1.515916e+18,apG36IgOcf,1,0,2.0
4,2021-02-24 15:59:50 UTC,view,1642613,2144415938450030080.0,25.56,1.515916e+18,7CZ3WuPoU3,1,0,2.0


In [11]:
df.isnull().sum()

event_time        0
event_type        0
product_id        0
category_id       0
price             0
user_id           0
user_session      0
user_score        0
user_purchase     0
price_category    0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883693 entries, 0 to 885123
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   event_time      883693 non-null  object 
 1   event_type      883693 non-null  object 
 2   product_id      883693 non-null  int64  
 3   category_id     883693 non-null  object 
 4   price           883693 non-null  float64
 5   user_id         883693 non-null  float64
 6   user_session    883693 non-null  object 
 7   user_score      883693 non-null  int64  
 8   user_purchase   883693 non-null  int64  
 9   price_category  883693 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 74.2+ MB


In [13]:
df['i_s'] =  df['user_score'] * df['price_category']*df['price']

In [14]:
# new dataframe grouped by user_id and product_id, for each product id sum of all user scores and user purchases.
df = df.groupby(['user_id','event_time'])[['event_type','product_id']].sum().reset_index()

std = MinMaxScaler(feature_range=(1,5))
std.fit(df['i_s'].values.reshape(-1,1))
df['interaction_score'] = std.transform(df['i_s'].values.reshape(-1,1))

# group['user_purchase'] = group['user_purchase'].apply(lambda x: 1 if x>1 else x)
# group['user_score'] = df['user_score'].apply(lambda x: 100 if x>100 else x)

#minmax scaler (1,5) applied on user score to get interaction score.
# std = MinMaxScaler(feature_range=(1,5))
# std.fit(group['user_score'].values.reshape(-1,1))
# group['interaction_score'] = std.transform(group['user_score'].values.reshape(-1,1))

# group = group.merge(df[['product_id','category_code','brand','price','price_category']].drop_duplicates('product_id'),on=['product_id'])

In [15]:
df

,user_id,event_time,i_s,product_id,interaction_score
0,1.515916e+18,2020-09-29 16:01:54 UTC,722.94,4005145,1.003071
1,1.515916e+18,2020-09-29 16:02:34 UTC,722.94,4005145,1.003071
2,1.515916e+18,2020-10-01 05:57:54 UTC,722.94,4005145,1.003071
3,1.515916e+18,2020-10-02 08:23:40 UTC,274.40,4099840,1.001165
4,1.515916e+18,2020-10-06 06:30:32 UTC,86.94,3506650,1.000368
...,...,...,...,...,...
864451,1.515916e+18,2021-02-28 23:54:18 UTC,64.44,3829355,1.000273
864452,1.515916e+18,2021-02-28 23:55:01 UTC,659.82,953226,1.002803
864453,1.515916e+18,2021-02-28 23:58:05 UTC,80.03,1715907,1.000339
864454,1.515916e+18,2021-02-28 23:58:09 UTC,129.84,4170534,1.000551


In [82]:
df.to_excel(r"C:\Users\Shibbs\Desktop\Praxis\CAPP\cwd\datasets\final\recommend.xlsx")

In [114]:
#minmax scaler (1,5) applied on user score to get interaction score.


# group = group.merge(df[['product_id','category_code','brand','price','price_category']].drop_duplicates('product_id'),on=['product_id'])

In [115]:
#df.sort_values(by='event_time', inplace=True)

In [33]:
df

,user_id,event_time,i_s,product_id
0,1.515916e+18,2020-09-29 16:01:54 UTC,3.0,4005145
1,1.515916e+18,2020-09-29 16:02:34 UTC,3.0,4005145
2,1.515916e+18,2020-10-01 05:57:54 UTC,3.0,4005145
3,1.515916e+18,2020-10-02 08:23:40 UTC,1.0,4099840
4,1.515916e+18,2020-10-06 06:30:32 UTC,3.0,3506650
...,...,...,...,...
864451,1.515916e+18,2021-02-28 23:54:18 UTC,2.0,3829355
864452,1.515916e+18,2021-02-28 23:55:01 UTC,3.0,953226
864453,1.515916e+18,2021-02-28 23:58:05 UTC,1.0,1715907
864454,1.515916e+18,2021-02-28 23:58:09 UTC,2.0,4170534


In [117]:
#df = df[['event_time','user_id', 'product_id', 'interaction_score']]

In [16]:
df['interaction_score'].std()

0.02778190267215901

In [17]:
df.interaction_score.min()

1.0

In [18]:
df.interaction_score.max()

5.0

In [19]:
df = df[['user_id', 'product_id', 'interaction_score']]

In [22]:
df

,user_id,product_id,interaction_score
0,1.515916e+18,4005145,1.003071
1,1.515916e+18,4005145,1.003071
2,1.515916e+18,4005145,1.003071
3,1.515916e+18,4099840,1.001165
4,1.515916e+18,3506650,1.000368
...,...,...,...
864451,1.515916e+18,3829355,1.000273
864452,1.515916e+18,953226,1.002803
864453,1.515916e+18,1715907,1.000339
864454,1.515916e+18,4170534,1.000551


In [25]:
reader = Reader(rating_scale=(1.0,5.0))
data = Dataset.load_from_df(df, reader)
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), 
                  KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), 
                  BaselineOnly()]:
    
 # Perform cross validation
 results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
 
 # Get results & append algorithm name
 tmp = pd.DataFrame.from_dict(results).mean(axis=0)
 tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
 benchmark.append(tmp)

KeyboardInterrupt: 

In [ ]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

In [70]:
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }

reader = Reader(rating_scale=(df.interaction_score.min(),df.interaction_score.max()))

data = Dataset.load_from_df(df, reader)
trainset, testset = train_test_split(data, test_size=0.3)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.0245


0.02452238104745542